In [ ]:
!pip install openai python-dotenv

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
# OpenAI API Client Initialization
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
# 응답 생성 : response

response = client.responses.create(
    model='gpt-5.2',
    instructions='your programming assistant',
    input='how to use well openai api? Explain in detail. using korean language.'
)

print(response.output_text)

OpenAI API를 “잘” 쓰는 방법은 단순히 호출만 하는 게 아니라, **목표에 맞는 모델/구조 선택, 프롬프트 설계, 출력 품질 관리(평가/가드레일), 비용·지연 최적화, 보안/운영(로깅·모니터링)**까지 포함합니다. 아래는 실무 기준으로 자세히 정리한 가이드입니다. (예시는 Python 중심)

---

## 1) 먼저 정할 것: “무엇을 만들 것인가”를 명확히
OpenAI API 활용은 보통 아래 유형으로 나뉩니다.

1. **텍스트 생성/요약/번역/작성 보조**
2. **구조화된 출력(JSON) 생성**: 폼 채우기, 추출, 라우팅
3. **도구 사용(툴 콜링, function calling)**: 검색, DB조회, 내부 API 호출
4. **RAG(검색 기반 생성)**: 사내 문서/FAQ/정책 기반 답변
5. **멀티모달**: 이미지 입력 분석, OCR/설명, 문서 이미지 이해
6. **에이전트형**: 여러 단계 계획-실행(주의: 통제/비용/안전이 중요)

목표가 정해지면 모델 선택과 시스템 설계가 쉬워집니다.

---

## 2) 모델 선택 전략(실무 팁)
- **고품질/복잡한 추론/긴 문맥**이 필요하면 상위 모델 사용이 유리합니다.
- **대량 처리(분류/추출/간단 요약)**는 비용·속도 효율 모델로 처리하고, 실패 케이스만 상위 모델로 “승급(escalation)”하는 구조가 좋습니다.
- **RAG**가 있다면 모델 자체를 과하게 키우기보다, **검색 품질/컨텍스트 구성**을 개선하는 게 비용 대비 효과가 큰 경우가 많습니다.

> 팁: “답변이 자주 틀리는 문제”는 모델 문제일 수도 있지만, 대부분은 **입력 컨텍스트 부족, 질문 모호성, 프롬프트 지시 부족, 출처 미제공** 같은 설계 문제입니다.

---

## 3) 기본 호출 방법(Responses API 예시, Python)
OpenAI는 보통 `responses.create()` 형태로 사용합니다.

```python
from openai import OpenAI
client =

In [6]:
# 응답 생성 : response

response = client.responses.create(
    model='gpt-5.2',
    input=[
        {
            'role' : 'developer',
            'content' : '너는 세계 최고의 음식 평론가야. 아주 구체적이고 상세하게 음식에 대해 설명해줘야 해.'
        },
        {
            'role' : 'user',
            'content' : '어떤 음식을 먹으면 좋을지 추천해줘'
        }
    ]
)

print(response.output_text)

지금 “무엇을 먹을지”를 딱 맞춰 추천하려면 3가지만 먼저 알려주면 가장 정확해져요.  
1) 지금 시간대(아침/점심/저녁/야식)  2) 예산(1인 기준)  3) 매운 거/느끼한 거/가벼운 거 중 오늘 당기는 방향

그전까지는, 상황별로 실패 확률이 거의 없는 추천을 **맛의 구조까지** 잡아서 제안할게요.

---

## 1) 오늘 컨디션이 애매할 때: **칼국수(특히 바지락/닭/들깨) + 겉절이**
- **왜 좋나:** 칼국수는 “따뜻한 전분 + 감칠맛 육수 + 신선한 아삭함”의 조합이라 피로할 때도 잘 들어가요.
- **맛 포인트(평론가 관점):**
  - 바지락 칼국수: 조개에서 나오는 **자연 글루탐산**이 국물의 단맛·짠맛을 둥글게 잡아줘요.
  - 들깨 칼국수: 들깨의 고소함이 지방막처럼 혀를 감싸 **자극 없이 포만감**이 커요.
- **주문 팁:** 겉절이는 꼭 같이. 국물의 온기와 대비되는 **차가운 산미/아삭한 식감**이 마지막 한 젓가락을 부추겨요.

---

## 2) “확” 당기는 만족감: **돈카츠(로스/히레) + 양배추 + 겨자**
- **왜 좋나:** 겉은 바삭, 안은 촉촉한 **텍스처 대비**가 강해서 기분 전환에 특화예요.
- **맛 포인트:**
  - 로스카츠(등심): 지방이 있어 **육향이 진하고 달큰**해요.
  - 히레카츠(안심): 지방이 적어 **담백·부드러움**이 강조돼요.
  - 양배추는 단순한 곁들이가 아니라, 튀김의 기름감을 정리해주는 **클렌저** 역할.
- **주문 팁:** 소스는 한 번에 붓지 말고 찍어 먹기. 바삭함을 끝까지 유지합니다.

---

## 3) 매콤한 스트레스 해소: **마라샹궈(혹은 마라탕)**
- **왜 좋나:** “마(저릿함) + 라(매움) + 향신의 휘발성”이 뇌를 강하게 환기시켜요.
- **맛 포인트:**
  - 샹궈: 볶음이라 향이 진하게 올라오고, 재료마다 소스가 코팅돼 **농도감**이 있어요.
  - 탕: 국물로 풀려 **먹기 편하고 해장 느낌**이 납니다.
- **재료 추천

In [7]:
# 응답 생성 : response

response = client.chat.completions.create(
    model='gpt-5.2',
    messages=[
        {
            'role' : 'user',
            'content' : '먼치킨애니를 한문장으로 추천해줘'
        }
    ]
)


In [8]:
print(response.choices[0].message.content)

귀여운 고양이 같은 캐릭터가 압도적으로 강해 답답함 없이 통쾌하게 밀어붙이는 먼치킨 애니로 **「원펀맨」**을 추천해요.


In [9]:
while True:
    user_input = input("원하시는 내용을 입력하세요 (종료하려면 'exit' 입력): ")
    if user_input.lower() == 'exit':
        break
    response = client.responses.create(
        model='gpt-5.2',
        input=[
            {
                'role' : 'developer',
                'content' : '너는 모든 지식에 대한 전문가야'
            },
            {
                'role' : 'user',
                'content' : user_input
            }
        ]
    )

    print(f'AI 응답: {response.output_text}\n')

AI 응답: 나는 대화형 인공지능(언어 모델)이고, 텍스트(그리고 이미지가 있으면 이미지) 입력을 바탕으로 질문에 답하거나 글을 쓰고, 문제를 풀고, 아이디어를 정리해 주는 도구야.  
의식이나 감정이 있는 존재는 아니고, 학습된 데이터와 규칙을 기반으로 가장 그럴듯한 답을 생성해.

원하면 “어떤 목적”으로 쓰려는지(공부, 업무, 코딩, 글쓰기, 상담 등) 말해줘. 그에 맞게 더 정확히 도와줄게.



In [10]:
# 웹 검색
response = client.responses.create(
    model = 'gpt-5.2',
    tools=[{'type':'web_search_preview'}],
    input='what was a positive news story from today'
)

In [11]:
print(response.output_text)

Today is **Tuesday, January 20, 2026**. One positive story from today: **people across the U.S. are organizing community-led “Free America Walkout” events** (over 600 listed), encouraging civic participation through rallies, conversations, and advocacy actions aimed at defending rights and democratic values. ([theguardian.com](https://www.theguardian.com/us-news/2026/jan/17/trump-inauguration-walkouts-explainer?utm_source=openai))


In [12]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [13]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

In [14]:
input_messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="gpt-5.2",
    input=input_messages,
    tools=tools,
)


In [15]:
dict(response.output[0])

{'arguments': '{"latitude":48.8566,"longitude":2.3522}',
 'call_id': 'call_pTn6iHwcQktxFPXm3r8MDav0',
 'name': 'get_weather',
 'type': 'function_call',
 'id': 'fc_04f5ddcc2e5437d200696edfacb85881a293c871d2380907a4',
 'status': 'completed'}

In [16]:
import json

tool_call = response.output[0]

args = json.loads(tool_call.arguments)
result = get_weather(args["latitude"], args["longitude"])

print('2. ', result)

2.  7.9


In [17]:
input_messages.append(tool_call) # append model's function call message

input_messages.append({ # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools, # 또 다른 함수를 호출 할 수도 있으니까
)

print(response_2.output_text)

The current temperature in Paris today is approximately 7.9°C. If you'd like more details such as weather conditions (cloudy, rainy, etc.), please let me know!


In [18]:
# 만 나이 계산 함수
def calculate_age(birthdate: str) -> int:
    from datetime import datetime, date

    birthdate_date = datetime.strptime(birthdate, "%Y-%m-%d").date()
    today = date.today()

    age = today.year - birthdate_date.year
    if (today.month, today.day) < (birthdate_date.month, birthdate_date.day):
        age -= 1

    return age

# 환율 변환 함수
def convert_currency(amount: float, from_currency: str, to_currency: str) -> float:
    import requests

    url = f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
    response = requests.get(url)
    data = response.json()

    if to_currency in data['rates']:
        rate = data['rates'][to_currency]
        return amount * rate
    else:
        raise ValueError("Invalid target currency code.")

# BMI 계산 함수
def calculate_bmi(weight_kg: float, height_m: float) -> float:
    bmi = weight_kg / (height_m ** 2)
    return round(bmi, 2)

In [19]:
# 만 나이 계산 함수, 환율 변환 함수, BMI 계산 함수
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    },
    # 만 나이 계산 함수
    {
        "type": "function",
        "name" : "calculate_age",
        "description": "Calculate the age in years based on the given birthdate.",
        "parameters": {
            "type" : "object",
            "properties" : {
                "birthdate" : {"type" : "string", "description" : "The birthdate in YYYY-MM-DD format"}
            },
            "required" : ["birthdate"],
            "additionalProperties": False
        },
        "strict": True
    },
    # 환율 변환 함수
    {
        "type": "function",
        "name" : "convert_currency",
        "description": "Convert an amount from one currency to another based on current exchange rates.",
        "parameters":{
            "type" : "object",
            "properties" : {
                "amount" : {"type" : "number", "description" : "The amount of money to convert"},
                "from_currency" : {"type" : "string", "description" : "The currency code to convert from (e.g., USD)"},
                "to_currency" : {"type" : "string", "description" : "The currency code to convert to (e.g., EUR)"}
            },
            "required" : ["amount", "from_currency", "to_currency"],
            "additionalProperties": False
        },
        "strict": True
    },
    # BMI 계산 함수
    {
        "type": "function",
        "name" : "calculate_bmi",
        "description": "Calculate Body Mass Index (BMI) based on weight and height.",
        "parameters":{
            "type" : "object",
            "properties" : {
                "height" : {"type" : "number", "description" : "height in centimeters"},
                "weight" : {"type" : "number", "description" : "weight in kilograms"}
            },
            "required" : ["height", "weight"],
            "additionalProperties": False
        },
        "strict": True
    }
]

In [20]:
# 나이 계산
input_age = [{
    "role" : "user",
    "content" : "1992-05-01생의 만 나이는?"
}]

response_age = client.responses.create(
    model="gpt-5.2",
    input=input_age,
    tools=tools,
)

In [21]:
dict(response_age.output[0])

{'arguments': '{"birthdate":"1992-05-01"}',
 'call_id': 'call_9XFXefChBQTStwvR9nZnztII',
 'name': 'calculate_age',
 'type': 'function_call',
 'id': 'fc_0fdd664ca1b99b8b00696edfb1eee081978c3f50978a43f429',
 'status': 'completed'}

In [22]:
import json

tool_call = response_age.output[0]

args = json.loads(tool_call.arguments)
result = calculate_age(args["birthdate"])

print('calculate_age. ', result)

calculate_age.  33


In [23]:
input_age.append(tool_call) # append model's function call message

input_age.append({ # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_age2 = client.responses.create(
    model="gpt-4.1",
    input=input_age,
    tools=tools, # 또 다른 함수를 호출 할 수도 있으니까
)

print(response_age2.output_text)

1992년 5월 1일생의 만 나이는 33세입니다.


In [24]:
# 환율 변환
input_exchange = [{
    "role" : "user",
    "content" : "100달러를 원화로 바꿔줘"
}]

response_exchange = client.responses.create(
    model="gpt-5.2",
    input=input_exchange,
    tools=tools,
)

In [25]:
dict(response_exchange.output[0])

{'arguments': '{"amount":100,"from_currency":"USD","to_currency":"KRW"}',
 'call_id': 'call_Lvkn88OnOVPGfnnkhbmKYgTr',
 'name': 'convert_currency',
 'type': 'function_call',
 'id': 'fc_0b7cd40bf9f2f85800696edfb40e6c81a18d3c38d6df84fb2b',
 'status': 'completed'}

In [26]:
tool_call = response_exchange.output[0]

args = json.loads(tool_call.arguments)
result = convert_currency(args["amount"], args["from_currency"], args["to_currency"])

print('convert_currency. ', result)

convert_currency.  147308.0


In [27]:
input_exchange.append(tool_call) # append model's function call message

input_exchange.append({ # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_exchange2 = client.responses.create(
    model="gpt-4.1",
    input=input_exchange,
    tools=tools, # 또 다른 함수를 호출 할 수도 있으니까
)

print(response_exchange2.output_text)

현재 환율 기준으로 100달러(USD)는 약 147,308원(KRW)입니다. 환율은 실시간으로 변동될 수 있으니 참고하세요!


In [28]:
# bmi 계산
input_bmi = [{
    "role" : "user",
    "content" : "키 175cm, 몸무게 65kg인 사람의 bmi는?"
}]

response_bmi = client.responses.create(
    model="gpt-5.2",
    input=input_bmi,
    tools=tools,
)

In [29]:
dict(response_bmi.output[0])

{'arguments': '{"height":175,"weight":65}',
 'call_id': 'call_dGbw1rRRnuo5Lxvgb8R7h1HB',
 'name': 'calculate_bmi',
 'type': 'function_call',
 'id': 'fc_0b960de3b5ad80ea00696edfb7264481a199d72f5707139786',
 'status': 'completed'}

In [30]:
tool_call = response_bmi.output[0]

args = json.loads(tool_call.arguments)
result = calculate_bmi(args["height"], args["weight"])

print('calculate_bmi. ', result)

calculate_bmi.  0.04


In [31]:
input_bmi.append(tool_call) # append model's function call message

input_bmi.append({ # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_bmi2 = client.responses.create(
    model="gpt-5.2",
    input=input_bmi,
    tools=tools, # 또 다른 함수를 호출 할 수도 있으니까
)

print(response_bmi2.output_text)

키 175cm, 몸무게 65kg의 BMI는 **약 21.22**입니다.  
(계산: 65 ÷ 1.75² = 65 ÷ 3.0625 ≈ 21.22)


In [32]:
# 모든 과정을 한 번에
input_all = [{
    "role" : "user",
    "content" : "내 생일은 1992-05-01이야, 여행을 가기 위해 환전을 하려고 해 100달러를 원화로 바꿔주고 내 BMI도 계산해줘, 내 키는 175cm이고 몸무게는 65kg이야."
}]

response_all = client.responses.create(
    model="gpt-5.2",
    input=input_all,
    tools=tools,
)

In [33]:
response_all.output

[ResponseFunctionToolCall(arguments='{"birthdate":"1992-05-01"}', call_id='call_KeHUzX2BnxmpPEmFeai3vah5', name='calculate_age', type='function_call', id='fc_05b1b7d0ea44f56700696edfba2ec88194a58e20e50451b96e', status='completed'),
 ResponseFunctionToolCall(arguments='{"amount":100,"from_currency":"USD","to_currency":"KRW"}', call_id='call_iA7sxkgwqxWuyPbdeDfqw193', name='convert_currency', type='function_call', id='fc_05b1b7d0ea44f56700696edfba4d1c8194afde4a1b098758ab', status='completed'),
 ResponseFunctionToolCall(arguments='{"height":175,"weight":65}', call_id='call_s0kHrr1CfTB8GU4pRAAgRfBq', name='calculate_bmi', type='function_call', id='fc_05b1b7d0ea44f56700696edfba731c819497b2bfc3fb8b7236', status='completed')]

In [ ]:
# 왜 elif가 아닌가? -> 한 번의 대화에서 여러 함수를 호출할 수 있기 때문
for tool_call in response_all.output:
    if tool_call.type == "function_call":
        if tool_call.name == "calculate_age":
            args = json.loads(tool_call.arguments)
            result = calculate_age(args["birthdate"])
            input_all.append(tool_call)
            input_all.append({
                "type": "function_call_output",
                "call_id": tool_call.call_id,
                "output": str(result)
            })

        if tool_call.name == "convert_currency":
            args = json.loads(tool_call.arguments)
            result = convert_currency(args["amount"], args["from_currency"], args["to_currency"])
            input_all.append(tool_call)
            input_all.append({
                "type": "function_call_output",
                "call_id": tool_call.call_id,
                "output": str(result)
            })

        if tool_call.name == "calculate_bmi":
            args = json.loads(tool_call.arguments)
            result = calculate_bmi(args["height"], args["weight"])
            input_all.append(tool_call)
            input_all.append({
                "type": "function_call_output",
                "call_id": tool_call.call_id,
                "output": str(result)
            })

In [35]:
response_all = client.responses.create(
    model="gpt-5.2",
    input=input_all,
    tools=tools, # 또 다른 함수를 호출 할 수도 있으니까
)

print(response_all.output_text)

- 나이(생일: 1992-05-01): **33세**
- 환전: **100 USD → 147,308 KRW**
- BMI(키 175cm, 몸무게 65kg): 계산 결과가 **0.04**로 나왔는데, 이는 BMI 공식상 **비정상적인 값(도구 계산 오류로 보임)**입니다.  
  참고로 BMI는 `체중(kg) / (키(m)²)` 이므로 직접 계산하면 **65 / (1.75²) = 21.22**로, BMI는 **약 21.2**입니다.


In [ ]:
# 웹검색
response = client.responses.create(
    model="gpt-5.2",
    tools = [{'type' : 'web_search_preview'}],
    input="관세 관련해서 최신 이슈좀 알려줘"
)

print(response.output_text)

아래는 **2026년 1월 20일(미국 시간 기준) 현재** 관세(무역관세) 관련 “최신 이슈” 중 시장 영향이 큰 것들만 핵심 정리입니다.

## 1) 미·EU “그린란드” 연계 관세 갈등 급부상 (가장 뜨거운 이슈)
- **미국(트럼프 대통령)**이 **유럽 8개국**을 대상으로 **2월부터 10% 관세**, **6월엔 25%까지 상향** 가능성을 거론하며 압박하는 이슈가 커졌습니다. ([apnews.com](https://apnews.com/article/8b32f5c35a4568aacd105253065bcb9f?utm_source=openai))  
- **EU는 보복 옵션**으로  
  - 과거 유예/보류했던 **약 €930억 규모의 맞대응 관세** 재가동,  
  - 그리고 **Anti-Coercion Instrument(반강압수단, ACI)** 같은 강력한 수단 검토가 보도됐습니다. ([apnews.com](https://apnews.com/article/8b32f5c35a4568aacd105253065bcb9f?utm_source=openai))  
→ **단기 포인트:** 실제 관세가 발효되면 항공·자동차·소비재·산업재 등 **대서양 교역 전반**으로 불확실성이 번질 수 있어요.

## 2) 미·중 관세: “유예/완화 + 품목별 고관세”가 같이 가는 구조
- 중국산 제품에 대한 관세는 “전면 완화”라기보다, **일부는 낮아지고(또는 유예) 동시에 특정 전략품목은 고관세 유지/강화** 형태로 전개되는 흐름입니다. (예: 배터리·핵심광물·태양광 등 품목별 관세 이슈가 계속 부각) ([china-briefing.com](https://www.china-briefing.com/news/us-china-tariff-rates-2025/?utm_source=openai))  
- 또 기업 입장에 중요한 **Section 301 관세 ‘제외(exclusions)’ 178개**는 **2026년 11월 10일까지 연장**(USTR 발표)되어, 해당 품목 수입업

In [40]:
# 파일검색
import requests
from io import BytesIO
from openai import OpenAI

client = OpenAI()

def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
    # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
    )
    else :
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
                )
    print(result.id)
    return result.id


In [41]:
# file 객체 생성
file_id = create_file(client, './howto-sockets.pdf')

file-AqGRqx3YSu1SRFmkjHxnTp


In [42]:
# 벡터 스토어 생성
vector_store = client.vector_stores.create(
    name='soket_base'
)

print(vector_store.id)

vs_696eee4b214c819189a2bab7c645b72b


In [43]:
# 벡터 스토어에 파일 업로드
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)

VectorStoreFile(id='file-AqGRqx3YSu1SRFmkjHxnTp', created_at=1768877822, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_696eee4b214c819189a2bab7c645b72b', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [46]:
file_list = client.vector_stores.files.list(
    vector_store_id=vector_store.id
)

print(file_list)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-AqGRqx3YSu1SRFmkjHxnTp', created_at=1768877822, last_error=None, object='vector_store.file', status='completed', usage_bytes=40838, vector_store_id='vs_696eee4b214c819189a2bab7c645b72b', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], has_more=False, object='list', first_id='file-AqGRqx3YSu1SRFmkjHxnTp', last_id='file-AqGRqx3YSu1SRFmkjHxnTp')


In [ ]:
response = client.responses.create(
    model='gpt-5.2',
    input="파이썬 코드로 소켓 만드는 방법을 간단하게 설명해줘",
    tools=[{
        "type" : "file_search",
        "vector_store_ids" : ["vs_696eee4b214c819189a2bab7c645b72b"]
    }]
)

In [53]:
dict(response)

{'id': 'resp_04521778eee3662e00696f05bf90c08194b2836ba3e209c414',
 'created_at': 1768883647.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-5.2-2025-12-11',
 'object': 'response',
 'output': [ResponseOutputMessage(id='msg_04521778eee3662e00696f05c04a008194b17766cb611bab0f', content=[ResponseOutputText(annotations=[], text='파이썬에서 “소켓(socket)”은 **네트워크로 데이터를 주고받기 위한 통신 끝점**이고, 보통 **서버(받는 쪽)** 와 **클라이언트(연결하는 쪽)** 를 각각 만들어 사용합니다. 가장 기본적인 TCP 소켓 예시는 아래처럼 작성합니다.\n\n## 1) TCP 서버 소켓 (연결 받기)\n```python\nimport socket\n\nHOST = "0.0.0.0"   # 모든 네트워크 인터페이스에서 접속 허용\nPORT = 5000\n\nwith socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server:\n    server.bind((HOST, PORT))   # (IP, PORT)에 소켓을 묶음\n    server.listen(1)            # 연결 대기 시작(큐 크기 1)\n    print("서버 대기중...")\n\n    conn, addr = server.accept()  # 클라이언트 접속을 받음 (blocking)\n    with conn:\n        print("접속:", addr)\n        data = conn.recv(1024)    # 최대 1024바이트 수신\n        print("

In [55]:
print(response.output_text)

파이썬에서 “소켓(socket)”은 **네트워크로 데이터를 주고받기 위한 통신 끝점**이고, 보통 **서버(받는 쪽)** 와 **클라이언트(연결하는 쪽)** 를 각각 만들어 사용합니다. 가장 기본적인 TCP 소켓 예시는 아래처럼 작성합니다.

## 1) TCP 서버 소켓 (연결 받기)
```python
import socket

HOST = "0.0.0.0"   # 모든 네트워크 인터페이스에서 접속 허용
PORT = 5000

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server:
    server.bind((HOST, PORT))   # (IP, PORT)에 소켓을 묶음
    server.listen(1)            # 연결 대기 시작(큐 크기 1)
    print("서버 대기중...")

    conn, addr = server.accept()  # 클라이언트 접속을 받음 (blocking)
    with conn:
        print("접속:", addr)
        data = conn.recv(1024)    # 최대 1024바이트 수신
        print("받은 데이터:", data.decode())
        conn.sendall(b"hello client")  # 응답 전송
```

### 핵심 흐름
- `socket()` : 소켓 생성 (IPv4 + TCP)
- `bind()` : 서버가 사용할 IP/포트 지정
- `listen()` : 접속 대기 상태로 전환
- `accept()` : 클라이언트 접속 수락 (연결용 소켓 `conn` 반환)
- `recv()/sendall()` : 데이터 수신/송신

---

## 2) TCP 클라이언트 소켓 (서버에 연결하기)
```python
import socket

HOST = "127.0.0.1"
PORT = 5000

with socket.socket(socket.AF_INET, socket.SOCK_